<a href="https://colab.research.google.com/github/Chavamanish/pneumonia_xray_detection_transferLearning/blob/main/XRAY_KERAS_TUNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_gen = ImageDataGenerator(rescale= 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_data = train_data_gen.flow_from_directory('/content/drive/MyDrive/chest_xray/train', target_size=(64,64), batch_size=32, class_mode='binary')

Found 5216 images belonging to 2 classes.


In [ ]:
test_data_gen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_data = test_data_gen.flow_from_directory('/content/drive/MyDrive/chest_xray/test', target_size=(64,64), batch_size=32, class_mode='binary')

Found 624 images belonging to 2 classes.


In [ ]:
validation_data_gen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_data = validation_data_gen.flow_from_directory('/content/drive/MyDrive/chest_xray/val', target_size=(64,64), batch_size=4, class_mode='binary')

Found 16 images belonging to 2 classes.


In [ ]:
!pip install keras-tuner -q

     |████████████████████████████████| 133 kB 5.2 MB/s 


In [ ]:
def build_model(hp):
    model = tf.keras.Sequential()

    model.add(
        tf.keras.layers.Conv2D(
            # Define the hyperparameter.
            filters=hp.Int("filters", min_value=32, max_value=512, step=32),
            kernel_size=hp.Int("kernel_size",min_value=3, max_value=11, step=2),
            activation="relu",
            padding="same",
            input_shape=[64,64,3]
            ))
    model.add(tf.keras.layers.MaxPool2D(pool_size=3, padding="same", strides=2))

    model.add(
        tf.keras.layers.Conv2D(
            # Define the hyperparameter.
            filters=hp.Int("filters", min_value=32, max_value=512, step=32),
            kernel_size=hp.Int("kernel_size",min_value=3, max_value=11, step=2),
            padding="same",
            activation="relu"
            ))
    model.add(tf.keras.layers.MaxPool2D(pool_size=3, padding="same", strides=2))

    model.add(
        tf.keras.layers.Conv2D(
            # Define the hyperparameter.
            filters=hp.Int("filters", min_value=32, max_value=512, step=32),
            kernel_size=hp.Int("kernel_size",min_value=3, max_value=11, step=2),
            padding="same",
            activation="relu"
            ))
    model.add(tf.keras.layers.MaxPool2D(pool_size=3, padding="same", strides=2))
    
    model.add(tf.keras.layers.Flatten())
    for i in range(hp.Int("FCLAYER",2,10)):
      model.add(tf.keras.layers.Dense(units=hp.Int(f'Layer{i}',min_value=32,max_value=512,step=32),activation=hp.Choice(f'Layer{i} activation',['relu','tanh'])))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"],
    )
    return model


In [ ]:
 from keras_tuner import RandomSearch
 from keras_tuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(hypermodel=build_model,objective="val_accuracy",max_trials=3,directory="my_dir",project_name="XRAYCLF")


In [ ]:
tuner_search.search(train_data,epochs=4,validation_data=validation_data)

Trial 3 Complete [00h 05m 24s]
val_accuracy: 0.5

Best val_accuracy So Far: 0.8125
Total elapsed time: 00h 20m 46s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        9472      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        200768    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        200768    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [ ]:
model.fit(train_data,epochs=10,validation_data=validation_data,initial_epoch=2)

Epoch 3/10
163/163 [==============================] - 75s 454ms/step - loss: 0.2937 - accuracy: 0.8790 - val_loss: 0.7907 - val_accuracy: 0.6250
Epoch 4/10
163/163 [==============================] - 73s 450ms/step - loss: 0.2295 - accuracy: 0.9059 - val_loss: 0.4453 - val_accuracy: 0.7500
Epoch 5/10
163/163 [==============================] - 74s 451ms/step - loss: 0.2053 - accuracy: 0.9185 - val_loss: 0.9702 - val_accuracy: 0.5625
Epoch 6/10
163/163 [==============================] - 73s 448ms/step - loss: 0.1820 - accuracy: 0.9285 - val_loss: 0.6271 - val_accuracy: 0.6250
Epoch 7/10
163/163 [==============================] - 73s 448ms/step - loss: 0.1630 - accuracy: 0.9350 - val_loss: 0.3746 - val_accuracy: 0.8125
Epoch 8/10
163/163 [==============================] - 73s 448ms/step - loss: 0.1734 - accuracy: 0.9339 - val_loss: 0.4172 - val_accuracy: 0.6875
Epoch 9/10
163/163 [==============================] - 73s 448ms/step - loss: 0.1607 - accuracy: 0.9367 - val_loss: 0.5330 - val_ac